In [1]:

!pip install --upgrade datasets fsspec transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 19.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.0
    Uninstalling transformers-4.57.0:
      Successfully uninstalled transformers-4.57.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling dat

In [2]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [3]:
from datasets import load_dataset
from transformers import BertTokenizer
# Load IMDB dataset and subset
dataset = load_dataset("imdb")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
train_dataset = dataset["train"].select(range(1000))
test_dataset = dataset["test"].select(range(500))

In [6]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})

**Tokenizewr**

In [7]:

# Initialize tokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

The raw text column (Each row in the IMDB dataset contains a review text)
It will pad each sentence to the same length (up to 256 tokens)
If the text is longer than 256 tokens, it will truncate (cut) it **bold text**

In [8]:

# Tokenization function
def tokenize_fn(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256) # each sentence will have 256 tokens


In [9]:

# Apply tokenization + rename + format in a single flow
def preprocess(ds):
    ds = ds.map(tokenize_fn, batched=True, remove_columns=["text"])  # remove raw text (saves memory)
    ds = ds.rename_column("label", "labels")
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    return ds

In [10]:
train_dataset = preprocess(train_dataset)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
train_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

In [13]:
test_dataset = preprocess(test_dataset)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

**Loading Model**

In [14]:
# 3. Model load
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./bert-finetuned-imdb",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    logging_dir="./logs",
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to="none"
)

In [16]:
# 5. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


In [17]:
# 6. Train
trainer.train()


Step,Training Loss


TrainOutput(global_step=125, training_loss=0.04788656997680664, metrics={'train_runtime': 55.9408, 'train_samples_per_second': 17.876, 'train_steps_per_second': 2.235, 'total_flos': 131555527680000.0, 'train_loss': 0.04788656997680664, 'epoch': 1.0})

In [18]:

!tensorboard --logdir=./logs

/usr/local/lib/python3.12/dist-packages/tensorboard/_vendor/bleach/sanitizer.py:292: SyntaxWarning: invalid escape sequence '\s'
  "[`\000-\040\177-\240\s]+",
/usr/local/lib/python3.12/dist-packages/tensorboard/_vendor/bleach/sanitizer.py:339: SyntaxWarning: invalid escape sequence '\s'
  style = re.compile('url\s*\(\s*[^\s)]+?\s*\)\s*').sub(' ', style)
/usr/local/lib/python3.12/dist-packages/tensorboard/_vendor/bleach/sanitizer.py:354: SyntaxWarning: invalid escape sequence '\s'
  if not re.match("^\s*([-\w]+\s*:[^:;]*(;\s*|$))*$", style):
/usr/local/lib/python3.12/dist-packages/tensorboard/_vendor/bleach/sanitizer.py:358: SyntaxWarning: invalid escape sequence '\w'
  for prop, value in re.findall('([-\w]+)\s*:\s*([^:;]*)', style):
2025-10-15 11:08:40.167653: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760526520.187823    6624 cuda_dnn.

In [19]:
trainer.save_model("./bert-finetuned-imdb")


In [20]:

tokenizer.save_pretrained("./bert-finetuned-imdb")

('./bert-finetuned-imdb/tokenizer_config.json',
 './bert-finetuned-imdb/special_tokens_map.json',
 './bert-finetuned-imdb/vocab.txt',
 './bert-finetuned-imdb/added_tokens.json')

In [21]:

# 7. Evaluate
metrics = trainer.evaluate()

In [22]:
print(metrics)

{'eval_loss': 0.0009491709643043578, 'eval_runtime': 6.3812, 'eval_samples_per_second': 78.355, 'eval_steps_per_second': 9.873, 'epoch': 1.0}


**Prediction After Fine TUning The base Model**

In [23]:

tokenizer = BertTokenizer.from_pretrained("/content/bert-finetuned-imdb")
model = BertForSequenceClassification.from_pretrained("/content/bert-finetuned-imdb")

In [24]:
from transformers import pipeline
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [25]:

# Predict
text = "This movie was amazing and I loved the acting!"
result = classifier(text)

**Pushing Hugging Face **

In [26]:
print(result)  # Example: [{'label': 'POSITIVE', 'score': 0.98}]

[{'label': 'LABEL_0', 'score': 0.9966018199920654}]


In [29]:
from huggingface_hub import notebook_login
notebook_login()

In [30]:

from huggingface_hub import whoami
print(whoami())

{'type': 'user', 'id': '67af9f292a4cd18639677a89', 'name': 'Rasel2091', 'fullname': 'Mohammad Rasel Ahmed', 'email': 'rasel4897981@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/7c51c1032da9e989d0b5a70b68b8c282.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'NA-BERT', 'role': 'write', 'createdAt': '2025-10-15T11:19:50.326Z'}}}


In [33]:
tokenizer.push_to_hub("Rasel2091/na-bert-imdb2")

CommitInfo(commit_url='https://huggingface.co/Rasel2091/na-bert-imdb2/commit/0e27421feb33b8955e70c32966105436d2dc5c2e', commit_message='Upload tokenizer', commit_description='', oid='0e27421feb33b8955e70c32966105436d2dc5c2e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Rasel2091/na-bert-imdb2', endpoint='https://huggingface.co', repo_type='model', repo_id='Rasel2091/na-bert-imdb2'), pr_revision=None, pr_num=None)

**ROBUT training and Fine tuning of my model**

In [34]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    BertTokenizer,
    BertTokenizerFast,
    BertForSequenceClassification,
    BertForTokenClassification,
    BertForQuestionAnswering,
    get_linear_schedule_with_warmup ###### Gradually warms up then decays learning rate for stable BERT training.
)
from datasets import load_dataset
from sklearn.metrics import accuracy_score, classification_report, f1_score
import numpy as np
from tqdm import tqdm
from torch.optim import AdamW #Adam Optimizer with Weight Decay


In [35]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [36]:

class Basket:
    def __init__(self, fruits):
        self.fruits = fruits  # List of fruits

    def __len__(self):
        return len(self.fruits)  # Kitne fruits hai total

    def __getitem__(self, idx):
        return self.fruits[idx]  # Index se specific fruit nikalna

In [37]:

basket = Basket(["Apple", "Banana", "Mango"])

In [38]:

print(len(basket))
print(basket[0])
print(basket[0])

3
Apple
Apple


In [39]:

# Dataset class
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [40]:

# BERT Text Classifier
class BERTTextClassifier:
    """BERT for Text Classification (Sentiment, Spam etc.)"""

    def __init__(self, model_name='bert-base-uncased', num_classes=2, max_length=512):
        self.model_name = model_name
        self.num_classes = num_classes
        self.max_length = max_length

        self.tokenizer = BertTokenizerFast.from_pretrained(model_name)
        self.model = BertForSequenceClassification.from_pretrained(
            model_name, num_labels=num_classes
        )
        self.model.to(device)

    def load_imdb_data(self, sample_size=5000):

        """Load IMDb movie reviews dataset"""

        print("Loading IMDb dataset...")

        dataset = load_dataset("imdb")

        # Sample data for faster training
        train_indices = np.random.choice(len(dataset['train']),
                                       min(sample_size, len(dataset['train'])),
                                       replace=False)

        test_indices = np.random.choice(len(dataset['test']),
                                      min(sample_size//4, len(dataset['test'])),
                                      replace=False)

        # Convert numpy.int64 → int for indexing
        train_texts = [dataset['train'][int(i)]['text'] for i in train_indices]
        train_labels = [dataset['train'][int(i)]['label'] for i in train_indices]

        test_texts = [dataset['test'][int(i)]['text'] for i in test_indices]
        test_labels = [dataset['test'][int(i)]['label'] for i in test_indices]

        print(f"Train samples: {len(train_texts)}")
        print(f"Test samples: {len(test_texts)}")

        return train_texts, train_labels, test_texts, test_labels

    def train(self, train_texts, train_labels, epochs=1, batch_size=8, learning_rate=2e-5):

        """Train the text classifier"""

        train_dataset = TextClassificationDataset(
            train_texts, train_labels, self.tokenizer, self.max_length
        )

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        optimizer = AdamW(self.model.parameters(), lr=learning_rate, weight_decay=0.01)

        total_steps = len(train_loader) * epochs

        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=total_steps//10, num_training_steps=total_steps
        )

        self.model.train()

        for epoch in range(epochs):
            total_loss = 0
            progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}')

            for batch in progress_bar:
                optimizer.zero_grad()

                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )

                loss = outputs.loss
                total_loss += loss.item()

                loss.backward()

                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)

                optimizer.step()

                scheduler.step()

                progress_bar.set_postfix({'Loss': f'{loss.item():.4f}'})

            print(f'Epoch {epoch+1}, Average Loss: {total_loss/len(train_loader):.4f}')

    def evaluate(self, test_texts, test_labels, batch_size=8):

        """Evaluate the text classifier"""

        test_dataset = TextClassificationDataset(
            test_texts, test_labels, self.tokenizer, self.max_length
        )

        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        self.model.eval()

        predictions = []

        true_labels = []

        with torch.no_grad():
            for batch in tqdm(test_loader, desc='Evaluating'):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                preds = torch.argmax(logits, dim=1).cpu().numpy()

                predictions.extend(preds)
                true_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(true_labels, predictions)
        f1 = f1_score(true_labels, predictions, average='weighted')
        report = classification_report(true_labels, predictions,
                                     target_names=['Negative', 'Positive'])

        return accuracy, f1, report

    def predict(self, texts):

        """Predict sentiment for new texts"""
        predictions = []
        probabilities = []

        self.model.eval()

        for text in texts:
            encoding = self.tokenizer(
                text,
                truncation=True,
                padding='max_length',
                max_length=self.max_length,
                return_tensors='pt'
            )

            input_ids = encoding['input_ids'].to(device)
            attention_mask = encoding['attention_mask'].to(device)

            with torch.no_grad():
                outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
                pred = torch.argmax(logits, dim=1).cpu().numpy()[0]

                predictions.append(pred)
                probabilities.append(probs)

        return predictions, probabilities

In [41]:

import torch
from torch.utils.data import Dataset

class NERDataset(Dataset):
    """Dataset for Named Entity Recognition"""

    def __init__(self, tokens_list, labels_list, tokenizer, max_length=512):
        self.tokens_list = tokens_list
        self.labels_list = labels_list
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.tokens_list)

    def __getitem__(self, idx):
        tokens = self.tokens_list[idx]
        labels = self.labels_list[idx]

        # Tokenize with word-level alignment
        encoding = self.tokenizer(
            tokens,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            is_split_into_words=True,
            return_tensors='pt'
        )

        # Get word alignment for batch index 0
        word_ids = encoding.word_ids(batch_index=0)
        aligned_labels = []
        previous_word_idx = None

        for word_idx in word_ids:
            if word_idx is None:
                aligned_labels.append(-100)  # Ignore special tokens
            elif word_idx != previous_word_idx:
                aligned_labels.append(labels[word_idx] if word_idx < len(labels) else 0)
            else:
                aligned_labels.append(-100)  # Ignore subword tokens
            previous_word_idx = word_idx

        # Ensure aligned_labels length matches max_length
        if len(aligned_labels) < self.max_length:
            aligned_labels += [-100] * (self.max_length - len(aligned_labels))
        elif len(aligned_labels) > self.max_length:
            aligned_labels = aligned_labels[:self.max_length]

        return {
            'input_ids': encoding['input_ids'].squeeze(0),  # Shape: (max_length,)
            'attention_mask': encoding['attention_mask'].squeeze(0),  # Shape: (max_length,)
            'labels': torch.tensor(aligned_labels, dtype=torch.long)  # Shape: (max_length,)
        }


In [42]:
class BERTNERClassifier:
    """BERT for Named Entity Recognition"""

    def __init__(self, model_name='bert-base-uncased', num_labels=9, max_length=512):
        self.model_name = model_name
        self.num_labels = num_labels
        self.max_length = max_length

        self.tokenizer = BertTokenizerFast.from_pretrained(model_name)

        self.model = BertForTokenClassification.from_pretrained(
            model_name, num_labels=num_labels
        )
        self.model.to(device)

        # Label mapping
        self.labels = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG',
                       'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

    def load_wikiann_data(self, sample_size=1000):
        """Load WikiAnn (Wikipedia + Annotation) dataset"""
        print("Loading wikiann NER dataset...")
        dataset = load_dataset("wikiann", "en")

        train_indices = np.random.choice(len(dataset['train']),
                                       min(sample_size, len(dataset['train'])),
                                       replace=False)

        test_indices = np.random.choice(len(dataset['test']),
                                      min(sample_size//4, len(dataset['test'])),
                                      replace=False)

        # Convert numpy.int64 to int
        train_tokens = [dataset['train'][int(i)]['tokens'] for i in train_indices]
        train_labels = [dataset['train'][int(i)]['ner_tags'] for i in train_indices]

        test_tokens = [dataset['test'][int(i)]['tokens'] for i in test_indices]
        test_labels = [dataset['test'][int(i)]['ner_tags'] for i in test_indices]

        print(f"Train samples: {len(train_tokens)}")
        print(f"Test samples: {len(test_tokens)}")

        return train_tokens, train_labels, test_tokens, test_labels

    def train(self, train_tokens, train_labels, epochs=1, batch_size=8, learning_rate=2e-5):
        """Train the NER model"""
        train_dataset = NERDataset(
            train_tokens, train_labels, self.tokenizer, self.max_length
        )
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        optimizer = AdamW(self.model.parameters(), lr=learning_rate, weight_decay=0.01)
        total_steps = len(train_loader) * epochs
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=total_steps//10, num_training_steps=total_steps
        )

        self.model.train()

        for epoch in range(epochs):
            total_loss = 0
            progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}')

            for batch in progress_bar:
                optimizer.zero_grad()

                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )

                loss = outputs.loss
                total_loss += loss.item()

                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()

                progress_bar.set_postfix({'Loss': f'{loss.item():.4f}'})

            print(f'Epoch {epoch+1}, Average Loss: {total_loss/len(train_loader):.4f}')

    def evaluate(self, test_tokens, test_labels, batch_size=8):
        """Evaluate the NER model"""
        test_dataset = NERDataset(
            test_tokens, test_labels, self.tokenizer, self.max_length
        )
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        self.model.eval()
        predictions = []
        true_labels = []

        with torch.no_grad():
            for batch in tqdm(test_loader, desc='Evaluating'):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                preds = torch.argmax(logits, dim=2).cpu().numpy()
                labels = labels.cpu().numpy()

                # Collect valid predictions (ignore -100)
                for i in range(preds.shape[0]):
                    for j in range(preds.shape[1]):
                        if labels[i][j] != -100:
                            predictions.append(preds[i][j])
                            true_labels.append(labels[i][j])

        accuracy = accuracy_score(true_labels, predictions)
        f1 = f1_score(true_labels, predictions, average='weighted')

        return accuracy, f1

    def predict(self, tokens_list):
        """Predict NER tags for new tokens"""
        predictions = []

        self.model.eval()
        for tokens in tokens_list:
            encoding = self.tokenizer(
                tokens,
                truncation=True,
                padding='max_length',
                max_length=self.max_length,
                return_tensors='pt',
                is_split_into_words=True
            )

            input_ids = encoding['input_ids'].to(device)
            attention_mask = encoding['attention_mask'].to(device)

            with torch.no_grad():
                outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                preds = torch.argmax(logits, dim=2).cpu().numpy()[0]

                # Get alignment for original words
                word_ids = encoding.word_ids(batch_index=0)
                token_predictions = []
                previous_word_idx = None

                for i, word_idx in enumerate(word_ids):
                    if word_idx is not None and word_idx != previous_word_idx:
                        if word_idx < len(tokens):
                            token_predictions.append(self.labels[preds[i]])
                    previous_word_idx = word_idx

                predictions.append(token_predictions)

        return predictions

In [43]:
class QADataset(Dataset):
    """Dataset for Question Answering (SQuAD-style)"""

    def __init__(self, questions, contexts, answers, tokenizer, max_length=512):
        self.questions = questions
        self.contexts = contexts
        self.answers = answers
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.contexts[idx]
        answer = self.answers[idx]  # dict: {'text': [...], 'answer_start': [...]}

        # Encode inputs with offsets to locate answer
        encoding = self.tokenizer(
            question,
            context,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_offsets_mapping=True,
            return_tensors='pt'
        )

        offset_mapping = encoding.pop("offset_mapping")[0]  # (max_length, 2)
        start_positions = torch.tensor(0, dtype=torch.long)
        end_positions = torch.tensor(0, dtype=torch.long)

        if answer and 'answer_start' in answer and answer['answer_start']:
            answer_start = answer['answer_start'][0]
            answer_text = answer['text'][0]
            answer_end = answer_start + len(answer_text)

            # Find token start/end matching answer char positions
            for idx, (start, end) in enumerate(offset_mapping):
                if start <= answer_start < end:
                    start_positions = torch.tensor(idx, dtype=torch.long)
                if start < answer_end <= end:
                    end_positions = torch.tensor(idx, dtype=torch.long)
                    break

        return {
            'input_ids': encoding['input_ids'].squeeze(0),      # (max_length,)
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'start_positions': start_positions,
            'end_positions': end_positions
        }


In [44]:
class BERTQuestionAnswering:
    """BERT for Question Answering (SQuAD)"""

    def __init__(self, model_name='bert-base-uncased', max_length=512):
        self.model_name = model_name
        self.max_length = max_length

        self.tokenizer = BertTokenizerFast.from_pretrained(model_name)
        self.model = BertForQuestionAnswering.from_pretrained(model_name)
        self.model.to(device)

    def load_squad_data(self, sample_size=2000):
        """Load and sample SQuAD dataset"""
        print("Loading SQuAD dataset...")
        dataset = load_dataset("squad")

        train_indices = np.random.choice(len(dataset['train']),
                                         min(sample_size, len(dataset['train'])),
                                         replace=False)
        val_indices = np.random.choice(len(dataset['validation']),
                                       min(sample_size//4, len(dataset['validation'])),
                                       replace=False)

        train_questions = [dataset['train'][int(i)]['question'] for i in train_indices]
        train_contexts = [dataset['train'][int(i)]['context'] for i in train_indices]
        train_answers = [dataset['train'][int(i)]['answers'] for i in train_indices]

        val_questions = [dataset['validation'][int(i)]['question'] for i in val_indices]
        val_contexts = [dataset['validation'][int(i)]['context'] for i in val_indices]
        val_answers = [dataset['validation'][int(i)]['answers'] for i in val_indices]

        print(f"Train samples: {len(train_questions)}")
        print(f"Validation samples: {len(val_questions)}")

        return (train_questions, train_contexts, train_answers,
                val_questions, val_contexts, val_answers)

    def train(self, questions, contexts, answers, epochs=1, batch_size=8, learning_rate=2e-5):
        """Train QA model using QADataset (offset mapping based)"""
        train_dataset = QADataset(questions, contexts, answers, self.tokenizer, self.max_length)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        optimizer = AdamW(self.model.parameters(), lr=learning_rate, weight_decay=0.01)

        total_steps = len(train_loader) * epochs

        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=total_steps//10, num_training_steps=total_steps
        )

        self.model.train()

        for epoch in range(epochs):
            total_loss = 0
            progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}')

            for batch in progress_bar:

                optimizer.zero_grad()

                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                start_positions = batch['start_positions'].to(device)
                end_positions = batch['end_positions'].to(device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    start_positions=start_positions,
                    end_positions=end_positions
                )

                loss = outputs.loss

                total_loss += loss.item()

                loss.backward()

                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)

                optimizer.step()

                scheduler.step()

                progress_bar.set_postfix({'Loss': f'{loss.item():.4f}'})

            print(f'Epoch {epoch+1}, Average Loss: {total_loss/len(train_loader):.4f}')

    def answer_question(self, question, context, max_answer_len=30):
        """Answer a single question given context"""

        encoding = self.tokenizer(
            question,
            context,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].to(device)

        attention_mask = encoding['attention_mask'].to(device)

        self.model.eval()

        with torch.no_grad():

            outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

            start_logits = outputs.start_logits

            end_logits = outputs.end_logits

            start_idx = torch.argmax(start_logits, dim=1).item()

            end_idx = torch.argmax(end_logits, dim=1).item()

            # Ensure valid span
            if end_idx < start_idx:
                end_idx = start_idx

            if (end_idx - start_idx) > max_answer_len:
                end_idx = start_idx + max_answer_len

            # Decode predicted tokens
            answer_tokens = input_ids[0][start_idx:end_idx+1]

            answer = self.tokenizer.decode(answer_tokens, skip_special_tokens=True).strip()

            return answer


In [45]:
def run_text_classification_demo():

    """Demo for text classification"""

    print("\n" + "="*60)
    print("TEXT CLASSIFICATION (Sentiment Analysis) DEMO")
    print("="*60)

    classifier = BERTTextClassifier(num_classes=2)

    # Load data
    train_texts, train_labels, test_texts, test_labels = classifier.load_imdb_data(sample_size=1000)

    # Show sample
    print(f"\nSample Review: {train_texts[0][:200]}...")
    print(f"Label: {'Positive' if train_labels[0] == 1 else 'Negative'}")

    # Train for 2 epochs (small for demo)
    classifier.train(train_texts, train_labels, epochs=1, batch_size=8)

    # Evaluate
    accuracy, f1, report = classifier.evaluate(test_texts, test_labels, batch_size=8)

    print(f"\nAccuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")

    # Test custom examples
    custom_reviews = [
        "This movie was fantastic! Amazing acting and great plot.",
        "Boring and terrible. Waste of time.",
        "Not bad, could be better though."
    ]

    predictions, probabilities = classifier.predict(custom_reviews)

    print(f"\nCustom Predictions:")

    for text, pred, prob in zip(custom_reviews, predictions, probabilities):

        sentiment = "Positive" if pred == 1 else "Negative"

        confidence = prob[pred] * 100

        print(f"'{text[:50]}...' -> {sentiment} ({confidence:.1f}%)")


In [46]:
def run_ner_demo():

    """Demo for Named Entity Recognition"""

    print("\n" + "="*60)
    print("NAMED ENTITY RECOGNITION DEMO")
    print("="*60)

    ner_model = BERTNERClassifier(num_labels=9)

    # Load small subset for demo (fast training)
    train_tokens, train_labels, test_tokens, test_labels = ner_model.load_wikiann_data(sample_size=500)

    # Show sample
    print(f"\nSample tokens: {train_tokens[0][:10]}")

    label_names = [ner_model.labels[l] if l < len(ner_model.labels) else "O" for l in train_labels[0][:10]]

    print(f"Sample labels: {label_names}")

    # Train for 2 epochs
    ner_model.train(train_tokens, train_labels, epochs=1, batch_size=8)

    # Evaluate
    accuracy, f1 = ner_model.evaluate(test_tokens, test_labels, batch_size=8)
    print(f"\nAccuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")

    # Custom examples
    custom_sentences = [
        ["John", "Smith", "works", "at", "Google", "in", "California"],
        ["Apple", "Inc.", "was", "founded", "by", "Steve", "Jobs"]
    ]

    predictions = ner_model.predict(custom_sentences)
    print(f"\nCustom Predictions:")
    for tokens, preds in zip(custom_sentences, predictions):
        print("Tokens:", tokens)
        print("Labels:", preds)
        print()

In [47]:
def run_qa_demo():

    """Demo for Question Answering"""

    print("\n" + "="*60)
    print("QUESTION ANSWERING DEMO")
    print("="*60)

    qa_model = BERTQuestionAnswering()

    # Load small subset (for speed)
    (train_questions, train_contexts, train_answers,
     val_questions, val_contexts, val_answers) = qa_model.load_squad_data(sample_size=500)

    # Safe sample print
    ans_text = train_answers[0]['text'][0] if train_answers[0]['text'] else 'No answer'

    print(f"\nSample Question: {train_questions[0]}")

    print(f"Sample Context: {train_contexts[0][:200]}...")

    print(f"Sample Answer: {ans_text}")

    # Train model (2 epochs for demo)
    qa_model.train(train_questions, train_contexts, train_answers, epochs=1, batch_size=4)

    # Test on custom questions
    print(f"\nCustom Q&A Examples:")

    test_cases = [
        {
            "question": "What is the capital of France?",
            "context": "France is a country in Europe. Paris is the capital and largest city of France. The city is known for the Eiffel Tower and the Louvre Museum."
        },
        {
            "question": "Who founded Apple?",
            "context": "Apple Inc. is an American technology company. It was founded by Steve Jobs, Steve Wozniak, and Ronald Wayne in 1976. The company is known for products like iPhone and Mac."
        }
    ]

    for case in test_cases:
        answer = qa_model.answer_question(case["question"], case["context"], max_answer_len=30)
        print(f"Q: {case['question']}")
        print(f"A: {answer}")
        print()


In [48]:
print("BERT Multi-Task Demo")
print("Choose a task to run:")
print("1. Text Classification (Sentiment Analysis)")
print("2. Named Entity Recognition (NER)")
print("3. Question Answering")
print("4. Run All Tasks")

choice = input("\nEnter your choice (1-4): ").strip()

try:
    if choice == "1":
        run_text_classification_demo()
    elif choice == "2":
        run_ner_demo()
    elif choice == "3":
        run_qa_demo()
    elif choice == "4":
        run_text_classification_demo()
        run_ner_demo()
        run_qa_demo()
    else:
        print("Invalid choice! Please run again.")
except Exception as e:
    print("\n--- ERROR OCCURRED ---")
    print(f"Error: {e}")
    print("\nMake sure you have:")
    print("1. Installed required packages:")
    print("   pip install torch transformers datasets scikit-learn tqdm numpy")
    print("2. Loaded all classes & dataset helpers (BERTTextClassifier, BERTNERClassifier, BERTQuestionAnswering, TextClassificationDataset, NERDataset, QADataset)")
    print("3. Using the fixed versions (with int casting, offset_mapping for QA, word_ids fix for NER)")


BERT Multi-Task Demo
Choose a task to run:
1. Text Classification (Sentiment Analysis)
2. Named Entity Recognition (NER)
3. Question Answering
4. Run All Tasks

Enter your choice (1-4): 4

TEXT CLASSIFICATION (Sentiment Analysis) DEMO


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading IMDb dataset...
Train samples: 1000
Test samples: 250

Sample Review: Dumb is as dumb does, in this thoroughly uninteresting, supposed black comedy. Essentially what starts out as Chris Klein trying to maintain a low profile, eventually morphs into an uninspired version...
Label: Negative


Epoch 1/1: 100%|██████████| 125/125 [01:30<00:00,  1.38it/s, Loss=0.5499]


Epoch 1, Average Loss: 0.5127


...

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Train samples: 500
Validation samples: 125

Sample Question: What was the grand total of German casualties and those missing in action?
Sample Context: The casualties were horrible, especially for the attacking Prussian forces. A grand total of 20,163 German troops were killed, wounded or missing in action during the August 18 battle. The French loss...
Sample Answer: 20,163


Epoch 1/1: 100%|██████████| 125/125 [00:53<00:00,  2.36it/s, Loss=4.0634]

Epoch 1, Average Loss: 4.9939

Custom Q&A Examples:
Q: What is the capital of France?
A: louvre

Q: Who founded Apple?
A: iphone

